In [2]:
import os


In [3]:
%pwd

'f:\\OneDrive - MSFT\\wine_quality_ML\\Machine-Learning-End-to-End-Project\\research'

In [4]:
# go back one folder
os.chdir('../')

In [6]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class DataingestionConfig:
    root_dir:Path
    Source_URL: str
    local_data_files:Path
    unzip_dir: Path

In [5]:
from src.MLProject_WineQT.constants.const import *
from src.MLProject_WineQT.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
     def __init__(
         self,
         config_filepath = CONFIG_FILE_PATH,
         params_filepath = PARAMS_FILE_PATH,
         schema_filepath = SCHEMA_FILE_PATH):
         
         self.config = read_yaml(config_filepath)
         self.params = read_yaml(params_filepath)
         self.schema = read_yaml(schema_filepath)
         
         create_directories([self.config.artifacts_root])
         
     def get_data_ingestion_config(self) -> DataingestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        get_data_ingestion_config=DataingestionConfig(root_dir=config.root_dir,
                                                      Source_URL=config.source_URL,
                                                      local_data_files=config.local_data_file,
                                                      unzip_dir=config.unzip_dir)
        return get_data_ingestion_config
           
             

In [7]:
# update the component
import os
import urllib.request as request
import zipfile
from src.MLProject_WineQT.my_logging.loger import logger
from src.MLProject_WineQT.utils.common import get_size


In [8]:
class DataIngestion:
    def __init__(self,config: DataingestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_files):
            filename,header = request.urlretrieve(
                url=self.config.Source_URL,
                filename=self.config.local_data_files
            ) 
            logger.info(f"{filename} download with following info: /n {header}")
        else:
            logger.info(f"file already exist fo size: {get_size(Path(self.config.local_data_files))}")
            
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        if zipfile.is_zipfile(self.config.local_data_files):
            with zipfile.ZipFile(self.config.local_data_files, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
        else:
            raise zipfile.BadZipFile(f"{self.config.local_data_files} is not a valid zip file")

In [6]:
from src.MLProject_WineQT.constants.const import *
from src.MLProject_WineQT.utils.common import read_yaml, create_directories
import os
import urllib.request as request
import zipfile
from src.MLProject_WineQT.my_logging.loger import logger
from src.MLProject_WineQT.utils.common import get_size
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataingestionConfig:
    root_dir: Path
    Source_URL: str
    local_data_files: Path
    unzip_dir: Path

class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        # Create root directory
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataingestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        return DataingestionConfig(
            root_dir=Path(config.root_dir),
            Source_URL=config.source_URL,
            local_data_files=Path(config.local_data_file),
            unzip_dir=Path(config.unzip_dir)
        )

class DataIngestion:
    def __init__(self, config: DataingestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_files):
            try:
                filename, header = request.urlretrieve(
                    url=self.config.Source_URL,
                    filename=self.config.local_data_files
                )
                logger.info(f"{filename} downloaded with the following info: \n{header}")
                
                if not zipfile.is_zipfile(self.config.local_data_files):
                    raise zipfile.BadZipFile(f"{self.config.local_data_files} is not a valid zip file")
                    
            except Exception as e:
                logger.error(f"Error downloading file: {e}")
                raise
        else:
            logger.info(f"File already exists with size: {get_size(self.config.local_data_files)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        
        if zipfile.is_zipfile(self.config.local_data_files):
            with zipfile.ZipFile(self.config.local_data_files, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_files} to {unzip_path}")
        else:
            raise zipfile.BadZipFile(f"{self.config.local_data_files} is not a valid zip file")



In [8]:
# Pipeline
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-10-03 11:22:28,249: INFO: common: yaml file:config\config.yaml load successfuly]
[2024-10-03 11:22:28,257: INFO: common: yaml file:params.yaml load successfuly]
[2024-10-03 11:22:28,265: INFO: common: yaml file:schema.yaml load successfuly]
[2024-10-03 11:22:28,269: INFO: common: created directories at: artifacts]
[2024-10-03 11:22:28,273: INFO: common: created directories at: artifacts\data_ingestion]
[2024-10-03 11:22:28,277: INFO: 2429842997: File already exists with size: 164141 KB]


BadZipFile: artifacts\data_ingestion\data.zip is not a valid zip file

In [7]:
import os
import urllib.request as request
import zipfile
from pathlib import Path
from dataclasses import dataclass
from src.MLProject_WineQT.utils.common import read_yaml, create_directories
from src.MLProject_WineQT.my_logging.loger import logger

@dataclass(frozen=True)
class DataingestionConfig:
    root_dir: Path
    Source_URL: str
    local_data_files: Path
    unzip_dir: Path

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH):
        self.config = read_yaml(config_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataingestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        return DataingestionConfig(
            root_dir=config.root_dir,
            Source_URL=config.source_URL,
            local_data_files=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

class DataIngestion:
    def __init__(self, config: DataingestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_files):
            try:
                filename, header = request.urlretrieve(
                    url=self.config.Source_URL,
                    filename=self.config.local_data_files
            )
                logger.info(f"{filename} downloaded with the following info: \n{header}")

            # After downloading, validate if it's a valid zip file
                if not zipfile.is_zipfile(self.config.local_data_files):
                    raise zipfile.BadZipFile(f"{self.config.local_data_files} is not a valid zip file")
            except Exception as e:
                logger.error(f"Error downloading file: {e}")
                raise 
            else:
                logger.info(f"File already exists with size: {get_size(Path(self.config.local_data_files))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        if zipfile.is_zipfile(self.config.local_data_files):
            with zipfile.ZipFile(self.config.local_data_files, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
            logger.info(f"Extracted {self.config.local_data_files} to {unzip_path}")
        else:
            raise zipfile.BadZipFile(f"{self.config.local_data_files} is not a valid zip file")

# Usage
if __name__ == "__main__":
    try:
        config_manager = ConfigurationManager()
        data_ingestion_config = config_manager.get_data_ingestion_config()
        data_ingestion = DataIngestion(config=data_ingestion_config)
        data_ingestion.download_file()
        data_ingestion.extract_zip_file()
    except Exception as e:
        logger.error(f"An error occurred: {e}")


[2024-10-03 11:34:26,816: INFO: common: yaml file:config\config.yaml load successfuly]
[2024-10-03 11:34:26,819: INFO: common: created directories at: artifacts]
[2024-10-03 11:34:26,828: INFO: common: created directories at: artifacts/data_ingestion]
[2024-10-03 11:34:31,787: INFO: 2699888841: artifacts/data_ingestion/data.zip downloaded with the following info: 
Connection: close
Content-Length: 21952
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "d8ac884c0b43591e6fccccdf678b39bfabcbcbb00f3ba0dc2886bc75bc948168"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 69FC:3FD81C:E947BB:EF51E5:66FEE3B6
Accept-Ranges: bytes
Date: Thu, 03 Oct 2024 18:34:32 GMT
Via: 1.1 varnish
X-Served-By: cache-fjr990023-FJR
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1727980472.273305,VS0,VE399
Vary: Authorization,Ac